In [1]:

import sys

sys.path.append("..")
from data.database import SessionLocal
from data.data_models import Restaurant, Foods

from tqdm import tqdm

In [2]:

import os
import requests
from openai import OpenAI
from sqlalchemy.orm import Session
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env")

client = OpenAI()
def generate_and_save_image(prompt, filename):
    # Check if the image already exists
    if not os.path.exists(filename):
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )

        image_url = response.data[0].url
        image_response = requests.get(image_url)
        
        # Ensure the directory exists
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        
        with open(filename, 'wb') as f:
            f.write(image_response.content)
    else:
        print(f"Image already exists: {filename}")

def process_restaurants(db: Session):
    restaurants = db.query(Restaurant).all()
    print("Generating restaurant images...")
    for restaurant in tqdm(restaurants):
        try:
            prompt = "Photorealistic zoomed, food-related picture for restaurant thumbnail: " + restaurant.name
            filename = os.path.join('../../../frontend/src/assets/', restaurant.image)
            generate_and_save_image(prompt, filename)
        except Exception as e:
            print(e)

def process_foods(db: Session):
    foods = db.query(Foods).all()
    print("Generating food images...")
    for food in tqdm(foods):
        try:
            prompt = "Photorealistic zoomed picture of food: " + food.name
            filename = os.path.join('../../../frontend/src/assets/', food.image)
            generate_and_save_image(prompt, filename)
        except Exception as e:
            print(e)

# Get the database session
db = SessionLocal()

try:
    # Process restaurants and foods
    process_restaurants(db)
    process_foods(db)
except Exception as e:
    print(e)

finally:
    # Close the session
    db.close()